In [1]:
# 3rd-party libraries
import numpy as np
from scipy.optimize import minimize
from scipy.stats import entropy

# Pretty-print numpy arrays
def pprint(x):
    print(np.array_str(x, precision=4, suppress_small=True))

In [2]:
# Create data
np.random.seed(12345)

k = 10 # Number of parameters to be estimated
t = 8 # Number of observations

# Create true, unknown probabilities

alpha = np.repeat(k,k)
ptrue = np.random.dirichlet(alpha)

print("True Probabilities:")
pprint(ptrue)


X = np.random.normal(size=(t,k))
y = np.dot(X, ptrue)

True Probabilities:
[0.0724 0.0899 0.1373 0.1174 0.0981 0.1127 0.1052 0.0494 0.1161 0.1016]


In [3]:
def classical_me_scipy(y, X, prob0=None):
    k = X.shape[1]

    # Objective function (to be minimized)
    def objective(p):
        return -entropy(p)  # negative because scipy.optimize.minimize performs minimization

    # Constraints
    cons = ({'type': 'eq', 'fun': lambda p: np.dot(X, p) - y},
            {'type': 'eq', 'fun': lambda p: np.sum(p) - 1})

    # Bounds
    bounds = [(0, 1) for _ in range(k)]

    # Initial guess
    if prob0 is None:
        prob0 = np.full(k, 1/k)

    # Solving the problem
    result = minimize(objective, prob0, method='SLSQP', bounds=bounds, constraints=cons) #sequential least squares programming algorithm

    if not result.success:
        raise ValueError("The problem was found to be infeasible")

    return result.x

In [4]:
# Classical ME results
results_me_scipy = classical_me_scipy(y, X)
pprint(results_me_scipy)

[0.0893 0.08   0.1286 0.1114 0.0903 0.1268 0.1005 0.052  0.1134 0.1078]


In [4]:
def classical_ce_scipy(prior, y, X, prob0=None):
    k = X.shape[1]

    # Objective function (to be minimized)
    def objective(p):
        return entropy(p, prior)

    # Constraints and bounds are the same as ME
    cons = ({'type': 'eq', 'fun': lambda p: np.dot(X, p) - y},
            {'type': 'eq', 'fun': lambda p: np.sum(p) - 1})
    bounds = [(0, 1) for _ in range(k)]

    # Initial guess
    if prob0 is None:
        prob0 = np.full(k, 1/k)

    # Solving the problem
    result = minimize(objective, prob0, method='SLSQP', bounds=bounds, constraints=cons)

    if not result.success:
        raise ValueError("The problem was found to be infeasible")

    return result.x

In [5]:
# minimize cross-entropy

# Case A Uniform Priors CE = ME
prior1 = np.repeat(1/k, k)
results_ce1 = classical_ce_scipy(prior1, y, X)
pprint(results_ce1)

# Case B Random Incorrect Priors
prior2 = np.array([0.11, 0.10, 0.09, 0.07, 0.14, 0.13, 0.09, 0.11, 0.07, 0.09])
results_ce2 = classical_ce_scipy(prior2, y, X)
pprint(results_ce2)

# Case C Correct priors
prior3 = ptrue
results_ce3 = classical_ce_scipy(prior3, y, X)
pprint(results_ce3)

[0.0893 0.08   0.1286 0.1114 0.0903 0.1268 0.1005 0.052  0.1134 0.1078]
[0.1034 0.0717 0.1213 0.1063 0.0837 0.1386 0.0965 0.0543 0.1111 0.1131]
[0.0725 0.0898 0.1372 0.1174 0.098  0.1128 0.1052 0.0494 0.1161 0.1016]
